# agora_api

> SDK to interact with Agora private api

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
from fastcore.utils import *
from fastcore.xtras import dict2obj
from httpx import get as xget, post as xpost, put as xput
from fasthtml.common import *
from monsterui.core import *


In [4]:
#| export

base_url = 'https://zues.searchagora.com/api/v1'

In [5]:
#| export

def search_trial(query: str, # Search query text
                 price_range: List[int] = None, # Price range filter [min_price, max_price]
                 sort: str = None, # Sorting field: price:relevance
                 order: str = None): # Sorting order: asc or desc
    """
    Search for products using the trial endpoint.
    
    Args:
        query (str): Search query text
        price_range (list, optional): Price range filter [min_price, max_price]
        sort (str, optional): Sorting field: price:relevance
        order (str, optional): Sorting order: asc or desc
        
    Returns:
        dict: Search results with products matching the query
    
    Example:
        search_trial("shoes", [100, 1000], "price:relevance", "desc")
    """
    endpoint = f'{base_url}/search/trial'
    
    # Build query parameters
    params = {'q': query}
    
    if price_range: params['priceRange'] = price_range
    if sort: params['sort'] = sort
    if order: params['order'] = order
    
    # Make the request
    r = xget(endpoint, params=params)
    
    # Return parsed JSON response
    return r.json()

In [6]:

p = search_trial('headphones')['Products'][0]
p

{'name': 'Headphones',
 'storeName': 'The Baby Cubby',
 'brand': 'Tonies',
 '_id': '6674b4aab49534201189688d',
 'slug': 'tonies-headphones-08c2c9a1-b3af-4883-aebf-2a283a1fc0fd-1718924458056',
 'price': 29.99,
 'isVerified': False,
 'isBoosted': False,
 'source': 'shopify',
 'images': ['https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130',
  'https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-2.jpg?v=1637054132',
  'https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-3.jpg?v=1637054134',
  'https://cdn.shopify.com/s/files/1/0467/0649/1556/products/blue_8b41cdb7-1f12-4b3d-b43c-8aac607377f3.jpg?v=1637336366',
  'https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Pink-1.jpg?v=1637336366',
  'https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Reddd.jpg?v=1637336366'],
 'url': 'https://babycubby.com/products/tonies-headphones',
 'agoraScore': 

In [7]:
#| export

def text_search(token, query, count=20, page=1, price_range=None, sort=None, order=None, image_id=None):
    """
    Search for products with full functionality.
    
    Args:
        token (str): Authentication token for API access
        query (str): Search query text
        count (int, optional): Number of products per page (default: 20, max: 250)
        page (int, optional): Page number for pagination (default: 1)
        price_range (list, optional): Price range filter [min_price, max_price]
        sort (str, optional): Sorting field: price:relevance
        order (str, optional): Sorting order: asc or desc
        image_id (str, optional): Image search identifier
        
    Returns:
        dict: Search results with products matching the query
    
    Example:
        text_search("your_token", "red shoes", count=10, page=1, 
                   price_range=[50, 200], sort="price", order="asc")
    """
    endpoint = f'{base_url}/search'
    
    # Build query parameters
    params = {'q': query, 'count': count, 'page': page}
    
    if price_range: params['priceRange'] = price_range
    if sort: params['sort'] = sort
    if order: params['order'] = order
    if image_id: params['imageId'] = image_id
    
    # Set authorization header
    headers = {'Authorization': f'Bearer {token}'}
    
    # Make the request
    r = xget(endpoint, params=params, headers=headers)
    
    # Return parsed JSON response
    return dict2obj(r.json())

In [8]:
#| export

def get_product_detail(token, slug):
    """
    Retrieve detailed information about a specific product.
    
    Args:
        token (str): Authentication token for API access
        slug (str): The unique identifier of the product to retrieve
        
    Returns:
        dict: Detailed information about the requested product
    
    Example:
        get_product_detail("your_token", "calzuro-without-pistachio-eb12f468-48a2-48af-9f5e-3fda5f6c135c-1708446961787")
    """
    endpoint = f'{base_url}/product-detail'
    
    # Build query parameters
    params = {'slug': slug}
    
    # Set authorization header
    headers = {'Authorization': f'Bearer {token}'}
    
    # Make the request
    r = xget(endpoint, params=params, headers=headers)
    
    # Return parsed JSON response
    return dict2obj(r.json())


In [9]:

# get_product_detail(token, p.slug)

In [10]:
#| export

def create_cart(token, custom_user_id=None, items=None):
    """
    Create a new cart for a user.
    
    Args:
        token (str): Authentication token for API access
        custom_user_id (str, optional): Unique identifier for the user
        items (list, optional): List of items to add to the cart
            Each item should be a dict with:
            - variantId (int): Variant ID of the product
            - product (int/str): Product ID
            - quantity (int): Quantity of the product
            
    Returns:
        dict: Response with cart creation status
    
    Example:
        create_cart("your_token", "user123", [
            {"variantId": 123, "product": "678f71a9356a36f784ee2e88", "quantity": 1}
        ])
    """
    url = f'{base_url}/cart'
    
    # Set headers
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    # Add custom user ID if provided
    if custom_user_id:
        headers['customuserid'] = custom_user_id
    
    # Prepare request body
    data = {}
    if items:
        data['items'] = items
    
    # Make the request
    r = xpost(url, headers=headers, json=data)
    
    # Return parsed JSON response
    return dict2obj(r.json())

In [11]:
#| export

def add_to_cart(token, product_id, variant_id, quantity=1, custom_user_id=None):
    """
    Add a product to an existing cart.
    
    Args:
        token (str): Authentication token for API access
        product_id (str/int): ID of the product to add
        variant_id (str/int): Variant ID of the product
        quantity (int, optional): Quantity of the product (default: 1)
        custom_user_id (str, optional): Unique identifier for the user
        
    Returns:
        dict: Response with cart update status
    
    Example:
        add_to_cart("your_token", "678f71a9356a36f784ee2e88", "2061038485517", 2, "user123")
    """
    url = f'{base_url}/cart'
    
    # Set headers
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    # Add custom user ID if provided
    if custom_user_id:
        headers['customuserid'] = custom_user_id
    
    # Prepare request body
    data = {
        "product": {
            "product": product_id,
            "variantId": variant_id,
            "quantity": quantity
        }
    }
    
    # Make the PUT request
    r = xput(url, headers=headers, json=data)
    
    # Return parsed JSON response
    return dict2obj(r.json())

In [12]:
#| export
def create_order(token, encrypted_payment_info, shipping_address, current_user):
    """
    Create a new order from cart items.
    
    Args:
        token (str): Authentication token for API access
        encrypted_payment_info (str): Encrypted payment information
        shipping_address (dict): Dictionary containing shipping address details:
            - addressFirst (str): Street address
            - city (str): City name
            - state (str): State/province
            - country (str): Country name
            - addressName (str): Name associated with address
            - zipCode (str): Postal/ZIP code
        current_user (dict): Dictionary containing user information:
            - firstname (str): User's first name
            - lastname (str): User's last name
            - email (str): User's email address
            - _id (str): User's ID
            
    Returns:
        dict: Order creation response with success status and order ID
    
    Example:
        create_order(
            "your_token",
            "encrypted_data",
            {
                "addressFirst": "123 Main St",
                "city": "New York",
                "state": "NY",
                "country": "US",
                "addressName": "Home",
                "zipCode": "10001"
            },
            {
                "firstname": "John",
                "lastname": "Doe",
                "email": "john@example.com",
                "_id": "user123"
            }
        )
    """
    url = f'{base_url}/order'
    
    # Set headers
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    # Prepare request body
    data = {
        "encryptedPaymentInfo": encrypted_payment_info,
        "shippingAddress": shipping_address,
        "currentUser": current_user
    }
    
    # Make the POST request
    r = xpost(url, headers=headers, json=data)
    
    # Return parsed JSON response
    return dict2obj(r.json())

In [13]:
#| export

def track_order(token, order_id):
    """
    Track an existing order by its ID.
    
    Args:
        token (str): Authentication token for API access
        order_id (str): Unique identifier of the order to track
        
    Returns:
        dict: Detailed order information including items, status, and shipping details
    
    Example:
        track_order("your_token", "67c8577b3e370f07d12c7722")
    """
    url = f'{base_url}/order-tracking/{order_id}'
    
    # Set headers
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    # Make the GET request
    r = xget(url, headers=headers)
    
    # Return parsed JSON response
    return dict2obj(r.json())

In [14]:
#| export 

def refresh_token(refresh_token_str):
    """
    Refresh API key and token by providing a valid refresh token.
    
    Args:
        refresh_token_str (str): The refresh token to validate and retrieve
                                 a new API key and refresh token
        
    Returns:
        dict: Object containing new API key, refresh token, and expiration time
              with the following structure:
              {
                "status": "success",
                "data": {
                  "apiKey": "...",
                  "refreshToken": "...",
                  "expiresAt": "..."
                }
              }
    
    Raises:
        ValueError: If the refresh token is invalid or missing
    """
    endpoint = f'{base_url}/refresh-token'
    
    # Prepare request body
    data = {"refreshToken": refresh_token_str}
    
    # Make the POST request
    r = xpost(endpoint, json=data)
    response = dict2obj(r.json())
    
    # Check for errors
    if r.status_code != 200 or response.status == "error":
        raise ValueError(response.message)
        
    return response


## AI Agent

In [15]:
from claudette import *
os.environ['ANTHROPIC_LOG'] = 'debug'


In [16]:
chat = Chat(models[1], sp='You are a helpful assistant that can search for products in a store',
             tools=[search_trial])
# chat.toolloop('can you search for headp`hones in the price range of 100 to 200', trace_func=print)

In [17]:
import json
from fasthtml.common import *
from monsterui.all import *
from fasthtml.jupyter import *

In [18]:
#| export

def ProductCard(
    item: str # JSON string containing product details (`name`, `price`, `brand`, `storeName`, `agoraScore`, `images`, `_id`)
):
    """
    Renders a product card UI component displaying product details clearly and attractively.
    Expects `item` as a JSON-formatted string with these keys:
    - `name`: Product name (str)
    - `price`: Product price (float or str)
    - `brand`: Brand name (str)
    - `storeName`: Store name (str)
    - `agoraScore`: Product rating score out of 100 (int or float)
    - `images`: List of image URLs (list[str]), first image used as main display
    - `_id`: Unique identifier for the product (str)
    """
    item = dict2obj(json.loads(item))
    return Card(
        DivHStacked(
            # Left column with image
            Div(
                Img(src=item.images[0], alt=item.name, cls="max-h-52 object-contain")
            ),
            # Right column with product details
            DivVStacked(
                H3(item.name),
                Strong(f"${item.price}"),
                Div(f"Brand: {item.brand}"),
                Div(f"Store: {item.storeName}"),
                Div(f"Rating: {item.agoraScore}/100"),
                Button("View Details", id=f"view-{item._id}")
            )
        )
    )


In [19]:
from functools import partial
app, rt = fast_app(hdrs=Theme.blue.headers())
Show = partial(HTMX, app=app, link=True)
server = JupyUvi(app, port=8001)
data = search_trial('headphones', price_range=[100, 200])
item = data['Products'][0]
ProductCard(json.dumps(item))

```html
<div class="uk-card ">
  <div class="uk-card-body space-y-6">
    <div class="flex flex-row items-center space-x-4">
      <div>
<img src="https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130" alt="Headphones" class="max-h-52 object-contain">      </div>
      <div class="flex flex-col items-center space-y-4">
        <h3 class="uk-h3 ">Headphones</h3>
<strong class="font-bold ">$29.99</strong>        <div>Brand: Tonies</div>
        <div>Store: The Baby Cubby</div>
        <div>Rating: 92/100</div>
<button type="submit" id="view-6674b4aab49534201189688d" class="uk-btn uk-btn-default" name="view-6674b4aab49534201189688d">View Details</button>      </div>
    </div>
  </div>
</div>

```

In [20]:
Show(ProductCard(json.dumps(item)))

In [21]:
chat = Chat(models[1], sp='''You are a helpful assistant that can search for products in a store. 
            Whenever you need to display a product, use the ProductCard tool.''',
             tools=[search_trial, ProductCard])
chat.toolloop('can you search for headphones in the price range of 100 to 200', trace_func=print)

[{'role': 'user', 'content': 'can you search for headphones in the price range of 100 to 200'}, {'role': 'assistant', 'content': [TextBlock(citations=None, text="I'll search for headphones in the price range of $100 to $200 for you.", type='text'), ToolUseBlock(id='toolu_01PyPi77BGYsWt8B7ugdpNUN', input={'query': 'headphones', 'price_range': [100, 200]}, name='search_trial', type='tool_use')]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01PyPi77BGYsWt8B7ugdpNUN', 'content': "{'status': 'success', 'Products': [{'name': 'Headphones', 'storeName': 'The Baby Cubby', 'brand': 'Tonies', '_id': '6674b4aab49534201189688d', 'slug': 'tonies-headphones-08c2c9a1-b3af-4883-aebf-2a283a1fc0fd-1718924458056', 'price': 29.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130', 'https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-P

I only found one headphone option within your specified price range of &#36;100-&#36;200:

- Sonic Headphones (Black) by Suunto
- Price: &#36;150
- Store: TC Running Co
- Rating: 92/100

Would you like me to search for more headphone options with different price ranges, or would you like more information about this particular model?

<details>

- id: `msg_01G79UST3iu3kJj4h91ankYk`
- content: `[{'citations': None, 'text': 'I only found one headphone option within your specified price range of $100-$200:\n\n- Sonic Headphones (Black) by Suunto\n- Price: $150\n- Store: TC Running Co\n- Rating: 92/100\n\nWould you like me to search for more headphone options with different price ranges, or would you like more information about this particular model?', 'type': 'text'}]`
- model: `claude-3-7-sonnet-20250219`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 6167, 'output_tokens': 86}`

</details>

In [22]:
chat.h

[{'role': 'user',
  'content': 'can you search for headphones in the price range of 100 to 200'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="I'll search for headphones in the price range of $100 to $200 for you.", type='text'),
   ToolUseBlock(id='toolu_01PyPi77BGYsWt8B7ugdpNUN', input={'query': 'headphones', 'price_range': [100, 200]}, name='search_trial', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01PyPi77BGYsWt8B7ugdpNUN',
    'content': "{'status': 'success', 'Products': [{'name': 'Headphones', 'storeName': 'The Baby Cubby', 'brand': 'Tonies', '_id': '6674b4aab49534201189688d', 'slug': 'tonies-headphones-08c2c9a1-b3af-4883-aebf-2a283a1fc0fd-1718924458056', 'price': 29.99, 'isVerified': False, 'isBoosted': False, 'source': 'shopify', 'images': ['https://cdn.shopify.com/s/files/1/0467/0649/1556/products/Heaphones-PDP-Assets-Gray-1.jpg?v=1637054130', 'https://cdn.shopify.com/s/files/1/0467/0649/1556/p

In [23]:
#| hide
import nbdev; nbdev.nbdev_export()